# COMP0005 - GROUP COURSEWORK 2023-24
# Gesture Recognition via Convex Hull 

Use the cell below for all python code needed to realise the **Jarvis march algorithm** (including auxiliary points structures and functions needed by this algorithm - if any). The `jarvismarch()` function itself should take as input parameter a list of 2D points (`inputSet`), and reorientation the subset of such points (`outputSet`) that lie on the convex hull.

In [ ]:
# TODO: insertion sort is notably faster than quicksort and mergesort for small arrays, so it may be worth using it for small subhulls in Chan.
# TODO: add doc strings to almost everything and use MD blocks for more context around the code
# TODO: make sure to use randint and not random.uniform for integer values

In [ ]:
import math
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (16, 10)
plt.rcParams['font.size'] = 20 

# Utilities

class Point2D:
    """
    Represents a 2D point with x and y coordinates.
    """

    x: float
    y: float
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __lt__(self, other: 'Point2D') -> bool:
        # This is used to sort the points in the input set
        return self.x < other.x or (self.x == other.x and self.y < other.y)

    def __eq__(self, other: 'Point2D') -> bool:
        return self.x == other.x and self.y == other.y

    def __hash__(self) -> int:
        return hash((self.x, self.y))

    def __repr__(self) -> str:
        return f"({self.x}, {self.y})"
    
def distance_squared(p1: Point2D, p2: Point2D) -> float:
    """Calculates the squared Euclidean distance between two points."""
    return (p1.x - p2.x)**2 + (p1.y - p2.y)**2

def polar_angle(p0: Point2D, p: Point2D) -> float:
    """Computes the polar angle of p relative to reference point p0.""" 
    return math.atan2(p.y - p0.y, p.x - p0.x)

def leftmost_point(points: list[Point2D]) -> Point2D:
    """Finds the leftmost point (smallest x, then smallest y)."""
    return min(points)

def copy_points(points: list[Point2D]) -> list[Point2D]:
    """Creates a deep copy of a list of points."""
    return [point for point in points]

CLOCKWISE = 1
ANTI_CLOCKWISE = -1
COLINEAR = 0
def orientation(p: Point2D, q: Point2D, r: Point2D) -> int:
    """Determines orientation of three points (clockwise, counter-clockwise, or colinear)."""
    cross_product = (q.y - p.y) * (r.x - q.x) - (q.x - p.x) * (r.y - q.y)

    if cross_product == 0:
        return COLINEAR
    return CLOCKWISE if cross_product > 0 else ANTI_CLOCKWISE

def plot_points(input_set: list[Point2D], output_set: list[Point2D]):
    """Plots the input points and the computed convex hull using matplotlib."""
    xs = [p.x for p in input_set]
    ys = [p.y for p in input_set]
    plt.scatter(xs, ys, color='blue')  # Set the color of input points to blue
    xs = [p.x for p in output_set]
    ys = [p.y for p in output_set]
    # close the loop
    xs.append(output_set[0].x)
    ys.append(output_set[0].y)
    # Set the color of hull points to red with red lines and dots
    plt.plot(xs, ys, color='red', linestyle='-', marker='o')
    plt.show()

def get_median(times: list[float]) -> float:
    """Calculates the median value of a list of numbers."""
    n = len(times)
    if n % 2 == 0:
        return (times[n // 2 - 1] + times[n // 2]) / 2
    return times[n // 2]

In [ ]:
# Sorting functions

GREATER_THAN = 1
EQUAL = 0
LESS_THAN = -1
def compare(p0: Point2D, p1: Point2D, p2: Point2D) -> int:
    """
    Compares two points relative to a reference point for use in polar angle sorting.

    Args:
        p0: The reference point.
        p1: The first point to compare.
        p2: The second point to compare.

    Returns:
        GREATER_THAN (1) if p1 has a larger polar angle than p2 relative to p0.
        LESS_THAN (-1) if p1 has a smaller polar angle than p2 relative to p0.
        EQUAL (0) if p1 and p2 have the same polar angle or if the difference 
            falls within a tolerance due to floating-point inaccuracies.
    """
    # first compare by angle
    angle1 = polar_angle(p0, p1)
    angle2 = polar_angle(p0, p2)

    # tolerance of 1e-9 (to avoid floating point inaccuracies)
    if abs(angle1 - angle2) > 1e-9:
        return LESS_THAN if angle1 < angle2 else GREATER_THAN

    # if angles are equal, compare by distance_squared_squared
    sq_dist1 = distance_squared(p0, p1)
    sq_dist2 = distance_squared(p0, p2)

    if sq_dist1 < sq_dist2:
        return LESS_THAN
    if sq_dist1 > sq_dist2:
        return GREATER_THAN

    return EQUAL

def quick_sort(points: list[Point2D], p0: Point2D) -> list[Point2D]:
    """
    Sorts a list of points in-place by their polar angle relative to a reference point p0,
    using the QuickSort algorithm.
    https://en.wikipedia.org/wiki/Quicksort
    """
    def partition(points: list[Point2D], low: int, high: int) -> int:
        pivot = points[high]  
        i = low - 1 

        for j in range(low, high):
            if compare(p0, points[j], pivot) <= 0: # angle with points[j] is less than or equal to angle with pivot
                i += 1
                points[i], points[j] = points[j], points[i]

        points[i + 1], points[high] = points[high], points[i + 1]
        return i + 1

    def _quick_sort(points: list[Point2D], low: int, high: int):
        if low < high:
            pivot_index = partition(points, low, high)
            _quick_sort(points, low, pivot_index - 1)
            _quick_sort(points, pivot_index + 1, high)
        
    _quick_sort(points, 0, len(points) - 1)
    return points  
    
def merge_sort(points: list[Point2D], p0: Point2D) -> list[Point2D]:
    """
    Sorts a list of points by their polar angle relative to a reference point p0,
    using the MergeSort algorithm.
    https://en.wikipedia.org/wiki/Merge_sort
    """
    def merge(left: list[Point2D], right: list[Point2D]) -> list[Point2D]:
        result = []
        while left and right:
            if compare(p0, left[0], right[0]) <= 0: # angle with left[0] is less than or equal to angle with right[0]
                result.append(left.pop(0))
            else:
                result.append(right.pop(0))
        result.extend(left or right)
        return result
   
    if len(points) > 1:
        mid = len(points) // 2
        left = merge_sort(points[:mid], p0)
        right = merge_sort(points[mid:], p0)
        return merge(left, right)
    else:
        return points
    
def heap_sort(points: list[Point2D], p0: Point2D) -> list[Point2D]:
    """
    Sorts a list of points by their polar angle relative to a reference point p0,
    using the HeapSort algorithm.
    https://en.wikipedia.org/wiki/Heapsort
    """
    def heapify(points: list[Point2D], n: int, i: int):
        """
        Builds a max-heap from an array.

        Args:
            points: The array to be heapified.
            n: The size of the heap.
            i: The index of the parent node.
        """
        largest = i  # Initialize the index of the largest element as the root
        l = 2 * i + 1  # Index of the left child
        r = 2 * i + 2  # Index of the right child

        # Compare with the left child (if within bounds)
        if l < n and compare(p0, points[l], points[largest]) > 0:
            largest = l

        # Compare with the right child (if within bounds)
        if r < n and compare(p0, points[r], points[largest]) > 0:
            largest = r

        # If the root was not the largest, swap and recursively heapify
        if largest != i:
            points[i], points[largest] = points[largest], points[i]
            heapify(points, n, largest)

    def _heap_sort(points: list[Point2D]):
        """Performs heap sort on the given array."""
        n = len(points)

        # Build the max-heap
        for i in range(n // 2 - 1, -1, -1):
            heapify(points, n, i)

        # Extract elements one by one by swapping the root with the end and heapifying
        for i in range(n - 1, 0, -1):
            points[i], points[0] = points[0], points[i]
            heapify(points, i, 0)

    _heap_sort(points)
    return points


def sorted_by_polar_angle(
        p0: Point2D,
        input_set: list[Point2D],
        sorting_algorithm: callable
    ) -> list[Point2D]:
    """Sorts a list of points by polar angle using a provided sorting algorithm."""
    return sorting_algorithm(input_set, p0)

In [ ]:
def jarvis_march(input_set: list[Point2D]) -> list[Point2D]:
    """
    Returns the list of points that lie on the convex hull (jarvis march algorithm)
    https://en.wikipedia.org/wiki/Gift_wrapping_algorithm
    Args:
        input_set: The list of points to compute the convex hull for.

    Returns:
        A list of points that lie on the convex hull.
    """
    # Start with the leftmost point
    point_on_hull = leftmost_point(input_set)
    convex_hull = [] 

    while True:
        convex_hull.append(point_on_hull)
        # Start with the first point as the endpoint of the segment from point_on_hull
        end_point = input_set[0]

        for j in range(0, len(input_set)):
            # Check against all other points
            dir = orientation(convex_hull[-1], end_point, input_set[j])
            if (
                end_point == point_on_hull or 
                dir == ANTI_CLOCKWISE or
                (dir == COLINEAR and 
                    distance_squared(convex_hull[-1], input_set[j]) > distance_squared(convex_hull[-1], end_point)
                )
            ):  
                # Found a more counterclockwise point
                end_point = input_set[j]

        point_on_hull = end_point

        # check if we have wrapped around to the first point
        if end_point == convex_hull[0]:
            break

    return convex_hull

Use the cell below for all python code needed to realise the **Graham scan** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `grahamscan()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points that lie on the convex hull (`outputSet`).

In [ ]:
def graham_scan(
        input_set: list[Point2D],
        sorting_algorithm = merge_sort # default sorting algorithm 
    ) -> list[Point2D]:
    """
    Returns the list of points that lie on the convex hull (graham scan algorithm)
    https://en.wikipedia.org/wiki/Graham_scan

    Args:
        input_set: The list of points to compute the convex hull for.
        sorting_algorithm: The sorting algorithm to use for sorting the points by polar angle.

    Returns:
        A list of points that lie on the convex hull.
    """
    if len(input_set) < 3:
        return input_set

    p0 = min(input_set, key=lambda p: (p.y, p.x))
    convex_hull = []
    input_set = sorted_by_polar_angle(p0, input_set, sorting_algorithm)
    for i in range(len(input_set)):
        while (
            len(convex_hull) > 1 and
            orientation(convex_hull[-2], convex_hull[-1], input_set[i]) != ANTI_CLOCKWISE
        ):
            convex_hull.pop()
        convex_hull.append(input_set[i])
    return convex_hull

def graham_scan_quick_sort(input_set: list[Point2D]) -> list[Point2D]:
    """Computes the convex hull using the Graham scan algorithm with QuickSort."""
    return graham_scan(input_set, quick_sort)

def graham_scan_merge_sort(input_set: list[Point2D]) -> list[Point2D]:
    """Computes the convex hull using the Graham scan algorithm with MergeSort."""
    return graham_scan(input_set, merge_sort)

def graham_scan_heap_sort(input_set: list[Point2D]) -> list[Point2D]:
    """Computes the convex hull using the Graham scan algorithm with HeapSort."""
    return graham_scan(input_set, heap_sort)

Use the cell below for all python code needed to realise the **Chen's** algorithm (including auxiliary data structures and functions needed by this algorithm - if any). The `chen()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points that lie on the convex hull (`outputSet`).

In [ ]:
def partition_points(input_set: list[Point2D], partition_size: int) -> list[list[Point2D]]: 
    """Partitions the input set into smaller sets of size partition_size."""
    return [input_set[i:i + partition_size] for i in range(0, len(input_set), partition_size)]

def right_tangent(p: Point2D, hull: list[Point2D]) -> int:
    """Returns the index of the right tangent of a point p to a convex hull."""
    if len(hull) == 1:
        return 0
    
    left, right   = 0, len(hull) - 1
    left_prev_dir = orientation(p, hull[0], hull[-1])
    left_next_dir = orientation(p, hull[0], hull[(left + 1) % right])
    while left < right:
        mid = (left + right) // 2
        mid_prev_dir = orientation(p, hull[mid], hull[abs(mid - 1) % len(hull)])
        mid_next_dir = orientation(p, hull[mid], hull[(mid + 1) % len(hull)])
        mid_side_dir = orientation(p, hull[left],   hull[mid])
        if mid_prev_dir != ANTI_CLOCKWISE and mid_next_dir != ANTI_CLOCKWISE:
            # tangent touches mid -> found
            return mid
        elif mid_side_dir == CLOCKWISE and (left_next_dir == ANTI_CLOCKWISE or
                                      left_prev_dir == left_next_dir) or \
                mid_side_dir == ANTI_CLOCKWISE and mid_prev_dir == ANTI_CLOCKWISE:
            # focus on the left side of loop
            right = mid 
        else:
            # focus on the right side of loop
            left  = mid + 1 
            # switch clockwise to anti-clockwise or vice versa
            left_prev_dir = -mid_next_dir
            left_next_dir = orientation(p, hull[left], hull[(left + 1) % len(hull)])
    return left

def visualize_convex_hulls(partitions: list[list[Point2D]], convex_hulls: list[list[Point2D]]):
    """Visualizes the partitions and the computed convex hulls using matplotlib."""
    for i in range(len(partitions)):
        xs = [p.x for p in partitions[i]]
        ys = [p.y for p in partitions[i]]
        plt.scatter(xs, ys)
        xs = [p.x for p in convex_hulls[i]]
        ys = [p.y for p in convex_hulls[i]]
        # close the loop
        xs.append(convex_hulls[i][0].x)
        ys.append(convex_hulls[i][0].y)
        plt.plot(xs, ys)
    plt.show()

def get_hull_leftmost_point(hulls: list[list[Point2D]]) -> tuple[int, int]:
    """Returns the index of the hull and the index of the leftmost point in the hull -> (hull_index, point_index)."""
    min_x = math.inf
    min_x_index = 0
    min_x_hull_index = 0
    for i in range(len(hulls)):
        for j in range(len(hulls[i])):
            if hulls[i][j].x < min_x:
                min_x = hulls[i][j].x
                min_x_index = j
                min_x_hull_index = i
    return min_x_hull_index, min_x_index

def _chan(input_set: list[Point2D], m: int) -> list[Point2D] | None:
    """
    The core implementation of Chan's algorithm.

    Args:
        input_set: The list of input points.
        m: The initial partition size.

    Returns:
        A list of points representing the convex hull of the input set, or None if m is not large enough.
    """

    hulls = [graham_scan(partition) for partition in partition_points(input_set, m)]

    # merge the convex hulls
    convex_hull = []

    next_hull_index, next_point_index = get_hull_leftmost_point(hulls)
    
    for _ in range(m + 1):
        convex_hull.append((next_hull_index, next_point_index))
        next_point = hulls[next_hull_index][next_point_index]
        end_hull_index = next_hull_index
        end_point_index = (next_point_index - 1) % len(hulls[next_hull_index])

        for i in range(0, len(hulls)):
            if i == next_hull_index:
                continue
            j = right_tangent(next_point, hulls[i])
                
            p = hulls[i][j]
            end_point = hulls[end_hull_index][end_point_index]
            dir = orientation(next_point, end_point, p)
            if (
                end_point == next_point 
                or dir == ANTI_CLOCKWISE  
                or (dir == COLINEAR and 
                        distance_squared(next_point, p) > distance_squared(next_point, end_point)
                    )
            ):
                end_hull_index = i
                end_point_index = j

        next_hull_index = end_hull_index
        next_point_index = end_point_index

        if (next_hull_index, next_point_index) == convex_hull[0]:
            return [hulls[i][j] for i, j in convex_hull]
        
    return None

def chan(input_set: list[Point2D]) -> list[Point2D] | None:
    """
    Computes the convex hull of an input set of points using Chan's algorithm.

    Args:
        input_set: The list of input points.

    Returns:
        A list of points representing the convex hull of the input set, or None if the hull is not found.
    """
    n = len(input_set)

    if n < 3:
        return input_set
    
    m = 3
    while m < n:
        output_set = _chan(input_set, m)
        if output_set is not None:
            return output_set

        m *= m

    return _chan(input_set, n)

Use the cell below to implement the **synthetic data generator** needed by your experimental framework (including any auxiliary data structures and functions you might need - be mindful of code readability and reusability).

In [ ]:
import random
DEFAULT_MIN_X, DEFAULT_MAX_X = 0, 32767
DEFAULT_MIN_Y, DEFAULT_MAX_Y = 0, 32767

class TestDataGenerator():
    """
    Generates sets of 2D points for testing convex hull algorithms.
    Controls point distribution, range, and reproducibility (via seed).

    Includes methods for:
    * Colinear points (all on a line)
    * Uniformly distributed random points
    * Points forming a specific size convex hull with a specific number of total points

    Attributes:
        seed: The seed for the random number generator.
        min_x: The minimum x-coordinate for the points.
        max_x: The maximum x-coordinate for the points.
        min_y: The minimum y-coordinate for the points.
        max_y: The maximum y-coordinate for the points.
    """
    def __init__(
            self, 
            seed: int | None = None, 
            min_x: int = DEFAULT_MIN_X, max_x: int = DEFAULT_MAX_X, 
            min_y: int = DEFAULT_MIN_Y, max_y: int = DEFAULT_MAX_Y
        ):
        self._rng = random.Random(seed)
        self.min_x = min_x
        self.max_x = max_x
        self.min_y = min_y
        self.max_y = max_y

    def set_seed(self, seed: int) -> None:
        """Sets the seed for the random number generator."""
        self._rng.seed(seed)

    @staticmethod
    def _linear_space(start: int, end: int, num_points: int) -> list[float]:
        if num_points <= 0:
            raise ValueError("Number of points must be positive.")

        if num_points == 1:
            return [start]

        step_size = (end - start) / (num_points - 1)
        return [start + i * step_size for i in range(num_points)]

    def generate_colinear_points(self, test_size: int) -> list[Point2D]:
        # not possible to generate more points than the range if we only take integer values
        if test_size > (self.max_x - self.min_x + 1) or test_size > (self.max_y - self.min_y + 1):
            raise ValueError("The number of points should be less than or equal to the range of the points")
        
        xs = self._linear_space(self.min_x, self.max_x, test_size)
        slope = self._rng.choice([-1, 0, 1])
        intercept = self.max_y if slope == -1 else 0
        return [Point2D(x, slope * x + intercept) for x in xs]

    def generate_random_points_uniform(self, test_size: int) -> list[Point2D]:
        """Generates uniformly distributed random points within the specified range of the data generator."""
        if test_size > (self.max_x - self.min_x + 1) * (self.max_y - self.min_y + 1):
            raise ValueError("The number of points should be less than or equal to the range of the points")
        
        return [
            Point2D(self._rng.uniform(self.min_x, self.max_x), self._rng.uniform(self.min_y, self.max_y)) 
            for _ in range(test_size)
        ]
    
    def generate_points(self, num_convex_hull_vertices: int, num_points: int) -> list[Point2D]:
        """Generates points forming a convex hull with a specific number of vertices and total points."""

        if num_convex_hull_vertices > num_points:
            raise ValueError("The number of convex hull vertices should be less than or equal to the number of points")
        
        if num_convex_hull_vertices == 1:
            assert num_points == 1
            return [Point2D(self._rng.uniform(self.min_x, self.max_x), self._rng.uniform(self.min_y, self.max_y))]
        
        if num_convex_hull_vertices == 2:
            return self.generate_colinear_points(num_points)

        points: list[Point2D] = []
        # adding inscribed triangle
        points.append(Point2D(0, 2))
        points.append(Point2D(math.sqrt(3), -1))
        points.append(Point2D(-math.sqrt(3), -1))

        # adding the rest of the outer points
        RADIUS_OUTER = 2
        for _ in range(num_convex_hull_vertices - 3):
            angle = self._rng.uniform(0, 2 * math.pi)
            x = RADIUS_OUTER * math.cos(angle)
            y = RADIUS_OUTER * math.sin(angle)
            points.append(Point2D(x, y))
        
        # adding the inner points
        num_inner_points = num_points - num_convex_hull_vertices
        RADIUS_INNER = 1
        i = 0
        while i < num_inner_points:
            x = self._rng.uniform(-RADIUS_INNER, RADIUS_INNER)
            y = self._rng.uniform(-RADIUS_INNER, RADIUS_INNER)
            if x**2 + y**2 < 1: # inside the unit circle
                points.append(Point2D(x, y))
                i += 1
        
        # scale and translate the points to fit the range (and turn them into integers)
        x_min, x_max = -RADIUS_OUTER, RADIUS_OUTER
        y_min, y_max = -RADIUS_OUTER, RADIUS_OUTER
        
        for i in range(len(points)):
            points[i].x = self.min_x + (points[i].x - x_min) * (self.max_x - self.min_x) / (x_max - x_min)
            points[i].y = self.min_y + (points[i].y - y_min) * (self.max_y - self.min_y) / (y_max - y_min)

        # shuffle the points
        self._rng.shuffle(points)
        
        return points
    

Use the cell below to implement the requested **experimental framework** API.

In [ ]:
import timeit

class ExperimentalFramework():
    """
    Facilitates the experimental evaluation of convex hull algorithms.

    Features:
        * Tracks execution times and the number of computed hull vertices.
        * Provides plotting functions for analysis.
        * Compares algorithm performance.
        * Flexible configuration of experiments (independent variable, trials per run).


    Attributes:
        experiment_name: The name of the experiment.
        independent_variable: The independent variable for the experiment.
        algorithms: A dictionary of algorithm names and their corresponding functions.
        times: A dictionary of algorithm names and their (variable, execution times) tuples.
        num_hull_vertices: A dictionary of algorithm names and their (variable, number of hull vertices) tuples.
        trials_per_run: The number of trials to run for each configuration.
    """

    def __init__(self, experiment_name: str):
        self.experiment_name = experiment_name
        self.independent_variable = "Input size"
        self.algorithms: dict[str, callable] = {}
        self.times: dict[str, list[tuple[float, list[float]]]] = {}
        self.num_hull_vertices: dict[str, list[tuple[float, list[int]]]] = {}
        self.trials_per_run = 10

    def set_independent_variable(self, variable: str):
        self.independent_variable = variable

    def set_trials_per_run(self, trials: int):
        self.trials_per_run = trials

    def add_algorithm(self, name: str, algorithm):
        """Adds an algorithm to the framework."""
        self.algorithms[name] = algorithm
        self.times[name] = []
        self.num_hull_vertices[name] = []

    def run_from_input(self, variable: int, input_set: list[Point2D]):
        """
        Runs the registered algorithms on a specific input set and records timing information and the number of hull vertices.

        For each algorithm, multiple trials are performed to account for variations in execution time.  

        Args:
            variable: The current value of the independent variable (e.g., the input size).
            input_set: A list of Point2D objects representing the input for the algorithms.
        """
        for name in self.algorithms:
            times_taken = []
            num_hull_vertices = []
            for _ in range(self.trials_per_run):
                # points are copied to avoid having a modified input_set in following trials
                trial_set = copy_points(input_set)
                start_time = timeit.default_timer()
                convex_hull = self.algorithms[name](trial_set)
                end_time = timeit.default_timer()
                times_taken.append(end_time - start_time)
                num_hull_vertices.append(len(convex_hull))
    
            times_taken.sort()
            num_hull_vertices.sort()
            self.times[name].append((variable, times_taken))
            self.num_hull_vertices[name].append((variable, num_hull_vertices))

    def run_from_generator(
            self,
            variable: int, 
            data_generator: TestDataGenerator, 
            num_points: int, 
            num_convex_hull_vertices: int | None = None
        ):
        """
        Runs the registered algorithms on multiple randomly generated inputs and records timing information and the number of hull vertices.

        Utilizes a TestDataGenerator to create diverse sets of points, either purely random or with a specified number of convex hull vertices. 
        Algorithm performance is taken over multiple trials to account for variations in execution time.

        Args:
            variable: The current value of the independent variable.
            data_generator: A TestDataGenerator object used to create input sets.
            num_points: The total number of points to generate in each input set.
            num_convex_hull_vertices: If provided, the generator will create inputs with this number of points on the convex hull; otherwise, purely random point sets are used.
        """
        times_taken_per_algorithm = {name: [] for name in self.algorithms}
        hull_vertices_per_algorithm = {name: [] for name in self.algorithms}
        
        for _ in range(self.trials_per_run):
            if num_convex_hull_vertices is None:
                input_set = data_generator.generate_random_points_uniform(num_points)
            else: 
                input_set = data_generator.generate_points(num_convex_hull_vertices, num_points)
                
            for name in self.algorithms:
                trial_set = copy_points(input_set)
                start_time = timeit.default_timer()
                convex_hull = self.algorithms[name](trial_set)
                end_time = timeit.default_timer()
                times_taken_per_algorithm[name].append(end_time - start_time)
                hull_vertices_per_algorithm[name].append(len(convex_hull))

        for name in self.algorithms:
            times_taken = times_taken_per_algorithm[name]
            num_hull_vertices = hull_vertices_per_algorithm[name]
            times_taken.sort()
            num_hull_vertices.sort()
            self.times[name].append((variable, times_taken))
            self.num_hull_vertices[name].append((variable, num_hull_vertices))

    def _plot_values(self, values: dict[str, list[tuple[float, float]]], ylabel: str):
        for name in values:
            x = [p[0] for p in values[name]]
            y = [p[1] for p in values[name]]
            plt.plot(x, y, label=name)
        plt.xlabel(self.independent_variable)
        plt.ylabel(ylabel)
        plt.title(self.experiment_name)
        plt.legend()

    def plot_mean_times(self):
        plt.figure()
        means = {}
        for name in self.times:
            means[name] = [(p[0], sum(p[1]) / self.trials_per_run) for p in self.times[name]]
        self._plot_values(means, f"Mean time taken (s) (n={self.trials_per_run})")
        plt.show()

    def plot_median_times(self):
        plt.figure()
        medians = {}
        for name in self.times:
            medians[name] = [(p[0], get_median(p[1])) for p in self.times[name]]
        self._plot_values(medians, f"Median time taken (s) (n={self.trials_per_run})")
        plt.show()

    def plot_min_times(self, max_y: float | None = None):
        plt.figure()
        mins = {}
        for name in self.times:
            mins[name] = [(p[0], p[1][0]) for p in self.times[name]]
        self._plot_values(mins, f"Minimum time taken (s) (n={self.trials_per_run})")
        if max_y is not None:
            plt.ylim(0, max_y)
        plt.show()

    def _plot_boxplot(
            self, 
            values: 
            dict[str, list[tuple[float, list[float]]]], 
            ylabel: str,
            number_of_boxes: int = 10
        ):
        plt.figure()
        medians = {}
        for name in values:
            medians[name] = [(p[0], get_median(p[1])) for p in values[name]]
        self._plot_values(medians, "")
        for name in values:
            num_of_boxes = min(number_of_boxes, len(values[name]))
            spacing = len(values[name]) // num_of_boxes
            times = values[name][::spacing]
            xs = [p[0] for p in times]
            ys = [p[1] for p in times]

            box_size = (max(xs) - min(xs)) / (3 * num_of_boxes + 1)
            plt.boxplot(ys, positions=xs, showfliers=False, widths=box_size)

        plt.xlabel(self.independent_variable)
        plt.ylabel(ylabel)
        plt.title(self.experiment_name)
        plt.show()
            
    def plot_boxplot_time_vs_n(self, number_of_boxes: int = 10):
        self._plot_boxplot(self.times, f"Median time taken (s) (n={self.trials_per_run})", number_of_boxes)

    def plot_boxplot_hull_vertices_vs_n(self, number_of_boxes: int = 10):
        self._plot_boxplot(self.num_hull_vertices, f"Median number of hull vertices (n={self.trials_per_run})", number_of_boxes)

    def _plot_min_time_ratios(self, algorithm1: str, algorithm2: str, label: str | None = None):
        # computing the minimums
        mins1 = [(p[0], p[1][0]) for p in self.times[algorithm1]]
        mins2 = [(p[0], p[1][0]) for p in self.times[algorithm2]]
        x = [p[0] for p in mins1]
        y = [p[1] / mins2[i][1] for i, p in enumerate(mins1)]
        plt.plot(x, y, label=label if label is not None else f"{algorithm1} / {algorithm2}")

    @staticmethod
    def plot_multiple_experiments_min_time_ratios(
            experiments: list['ExperimentalFramework'], 
            algorithm1: str, 
            algorithm2: str
        ):
        plt.figure()
        for experiment in experiments:
            experiment._plot_min_time_ratios(algorithm1, algorithm2, experiment.experiment_name)
        
        # plot the line for the ratio of 1
        x = [p[0] for p in experiments[0].times[algorithm1]]
        y = [1 for _ in range(len(x))]
        plt.plot(x, y, label="1", linestyle="--", color="black")
        plt.xlabel(experiments[0].independent_variable)
        plt.ylabel(f"Minimum time ratio (n={experiments[0].trials_per_run})")
        plt.title(f"{algorithm1} / {algorithm2}")
        plt.legend()
        plt.show()

    # test one algorithm against another (used for debugging)
    @staticmethod
    def test_algorithm_vs_algorithm(
            algorithm1: callable, 
            algorithm2: callable, 
            max_num_points: int, 
            max_hull_vertices: int,
            data_generator: TestDataGenerator
        ):
        num_passes = 0
        num_trials = 0
        # TODO: make this start at 1 (fix the case of 2 for chan)
        for h in range(3, max_hull_vertices + 1):
            for n in range(h, max_num_points + 1):
                num_trials += 1

                input_set = data_generator.generate_points(h, n)
                output1 = algorithm1(copy_points(input_set))
                output2 = algorithm2(copy_points(input_set))
                output1.sort()
                output2.sort()
                if output1 != output2:
                    print(f"Test failed for h={h}, n={n}")
                    print(f"Output1: \n")
                    plot_points(input_set, output1)
                    print(f"Output2: \n")
                    plot_points(input_set, output2)
                else:
                    num_passes += 1

        print(f"Test passed for {num_passes}/{num_trials} trials")



Use the cell below to illustrate the python code you used to **fully evaluate** the three convex hull algortihms under considerations. The code below should illustrate, for example, how you made used of the **TestDataGenerator** class to generate test data of various size and properties; how you instatiated the **ExperimentalFramework** class to  evaluate each algorithm using such data, collect information about their execution time, plots results, etc. Any results you illustrate in the companion PDF report should have been generated using the code below.

In [ ]:
# Initialize the data generator
data_generator = TestDataGenerator(seed=42)

In [ ]:
# Experiment Graham Scan (Average Case)
# TODO: give more descriptive title to each experiment
experiment_graham_average = ExperimentalFramework("Graham Scan on Randomly (Uniformly) Generated Points")
experiment_graham_average.set_independent_variable("Number of points")
experiment_graham_average.set_trials_per_run(50)

# think about whether this should be in two differnet graphs or one graph
experiment_graham_average.add_algorithm("Graham Scan (Quick Sort)", graham_scan_quick_sort)
# TODO: make sure to mention why for future experiments, we will use merge sort because it always has O(nlogn) time complexity (not just on worst case)
experiment_graham_average.add_algorithm("Graham Scan (Merge Sort)", graham_scan_merge_sort)
experiment_graham_average.add_algorithm("Graham Scan (Heap Sort)", graham_scan_heap_sort)

start = 2
end = 1000000
step = 1000
plot_step = 10
data_generator.set_seed(42)
for n in range(start, end + step, step):
    experiment_graham_average.run_from_generator(n, data_generator, n)

    if (n - start) % plot_step == 0:
        print(f"Finished running for n={n}")
        experiment_graham_average.plot_boxplot_time_vs_n(8)

experiment_graham_average.plot_boxplot_time_vs_n(8)

In [ ]:
# Experiment Graham Scan (Worst Case)
experiment_graham_worst = ExperimentalFramework("Graham Scan on Worst Case (n=h) Points")
experiment_graham_worst.set_independent_variable("Number of points")
experiment_graham_worst.set_trials_per_run(50)

experiment_graham_worst.add_algorithm("Graham Scan (Quick Sort)", graham_scan_quick_sort)
experiment_graham_worst.add_algorithm("Graham Scan (Merge Sort)", graham_scan_merge_sort)
experiment_graham_worst.add_algorithm("Graham Scan (Heap Sort)", graham_scan_heap_sort)

start = 2
end = 1_000_000
step = 1000
plot_step = 1000
data_generator.set_seed(42)
for n in range(start, end + step, step):
    experiment_graham_worst.run_from_generator(n, data_generator, n, n)

    if (n - start) % plot_step == 0:
        print(f"Finished running for n={n}")
        experiment_graham_worst.plot_boxplot_time_vs_n(8)

experiment_graham_worst.plot_boxplot_time_vs_n(8)

In [ ]:
# Experiment Jarvis March (Average Case)
# TODO: give more descriptive title to each experiment
experiment_jarvis_average = ExperimentalFramework("Jarvis March on Randomly (Uniformly) Generated Points")
experiment_jarvis_average.set_independent_variable("Number of points")
experiment_jarvis_average.set_trials_per_run(50)

experiment_jarvis_average.add_algorithm("Jarvis March", jarvis_march)

start = 2
end = 10000
step = 10
plot_step = 1000
data_generator.set_seed(42)
for n in range(start, end + step, step):
    experiment_jarvis_average.run_from_generator(n, data_generator, n)

    if (n - start) % plot_step == 0:
        print(f"Finished running for n={n}")
        experiment_jarvis_average.plot_boxplot_time_vs_n(5)

experiment_jarvis_average.plot_boxplot_time_vs_n(5)

In [ ]:
# Experiment Jarvis March (Worst Case)
experiment_jarvis_worst = ExperimentalFramework("Jarvis March on Worst Case (n=h) Points")
experiment_jarvis_worst.set_independent_variable("Number of points")
experiment_jarvis_worst.set_trials_per_run(5)

experiment_jarvis_worst.add_algorithm("Jarvis March", jarvis_march)

start = 2
end = 10000
step = 10
plot_step = 1000
data_generator.set_seed(42)

for n in range(start, end + step, step):
    input_set = data_generator.generate_points(n, n)
    experiment_jarvis_worst.run_from_input(n, input_set)

    if (n - start) % plot_step == 0:
        print(f"Finished running for n={n}")
        experiment_jarvis_worst.plot_min_times()

experiment_jarvis_worst.plot_min_times()

In [ ]:
# Experiment Chan's Algorithm (Average Case)
# TODO: give more descriptive title to each experiment
experiment_chan_average = ExperimentalFramework("Chan's Algorithm on Randomly (Uniformly) Generated Points")
experiment_chan_average.set_independent_variable("Number of points")
experiment_chan_average.set_trials_per_run(100)

experiment_chan_average.add_algorithm("Chan's Algorithm", chan)

start = 2
end = 1000
step = 10
plot_step = 10
data_generator.set_seed(42)
for n in range(start, end + step, step):
    experiment_chan_average.run_from_generator(n, data_generator, n)

    if (n - start) % plot_step == 0:
        print(f"Finished running for n={n}")
        experiment_chan_average.plot_boxplot_time_vs_n(5)

experiment_chan_average.plot_boxplot_time_vs_n(10)

In [ ]:
# Experiment Chan's Algorithm (Worst Case)
experiment_chan_worst = ExperimentalFramework("Chan's Algorithm on Worst Case (n=h) Points")
experiment_chan_worst.set_independent_variable("Number of points")
experiment_chan_worst.set_trials_per_run(10)

experiment_chan_worst.add_algorithm("Chan's Algorithm", chan)

start = 2
end = 100
step = 10
plot_step = 10
data_generator.set_seed(42)
for n in range(start, end + step, step):
    input_set = data_generator.generate_points(n, n)
    experiment_chan_worst.run_from_input(n, input_set)

    if (n - start) % plot_step == 0:
        print(f"Finished running for n={n}")
        experiment_chan_worst.plot_min_times()

experiment_chan_worst.plot_min_times()

In [ ]:
# Comparing All Algorithms Together in Randomly (Uniformly) Generated Points
# TODO: give more descriptive title to each experiment
experiment_all_average = ExperimentalFramework("Comparing All Algorithms Together on Randomly (Uniformly) Generated Points")
experiment_all_average.set_independent_variable("Number of points")
experiment_all_average.set_trials_per_run(50)

experiment_all_average.add_algorithm("Jarvis March", jarvis_march)
# TODO: make sure to mention that the Graham Scan used inside Chan's Algorithm is the same as the one it is being compared to
experiment_all_average.add_algorithm("Graham Scan", graham_scan_merge_sort)
experiment_all_average.add_algorithm("Chan's Algorithm", chan)

start = 2
end = 1_000_000
step = 1000
plot_step = 1000
data_generator.set_seed(42)
for n in range(start, end + step, step):
    experiment_all_average.run_from_generator(n, data_generator, n)

    if (n - start) % plot_step == 0:
        print(f"Finished running for n={n}")
        experiment_all_average.plot_boxplot_time_vs_n(6)

experiment_all_average.plot_boxplot_time_vs_n(6)

In [ ]:
# Analysis of Worst Case Time Complexity of All Algorithms
# TODO: give more descriptive title to each experiment
experiment_worst_case = ExperimentalFramework("Worst Case (n=h) Time Complexity of All Algorithms")
experiment_worst_case.set_independent_variable("Number of points")
experiment_worst_case.set_trials_per_run(10)

experiment_worst_case.add_algorithm("Jarvis March", jarvis_march)
experiment_worst_case.add_algorithm("Graham Scan", graham_scan_merge_sort)
experiment_worst_case.add_algorithm("Chan's Algorithm", chan)

start = 2
end = 1000
step = 100
plot_step = 100
data_generator.set_seed(42)
for n in range(start, end + step, step):
    input_set = data_generator.generate_points(n, n)
    experiment_worst_case.run_from_input(n, input_set)

    if (n - start) % plot_step == 0:
        print(f"Finished running for n={n}")
        experiment_worst_case.plot_min_times(0.1)

experiment_worst_case.plot_min_times(0.1)

In [ ]:
# Analysis to see when Chan's Algorithm is better than Graham Scan
# we fix h to multiple different values (e.g. 3, 5, 10, 20, 50, 100) and vary n

# Experiment Chan's Algorithm vs Graham Scan
chans_algorithm = "Chan's Algorithm"
graham_scan_algorithm = "Graham Scan"
hs = [3, 10, 20] #, 10, 20, 50, 100]
experiments = {h: ExperimentalFramework(f"Chan's Algorithm / Graham Scan (h={h})") for h in hs}
for h in hs:
    experiments[h].set_independent_variable("Number of points")
    experiments[h].set_trials_per_run(20)
    experiments[h].add_algorithm(chans_algorithm, chan)
    experiments[h].add_algorithm(graham_scan_algorithm, graham_scan_merge_sort)

start = 0
end = 10_000_000
step = 100
plot_step = 1000
data_generator.set_seed(42)
for n in range(start, end + step, step):
    for h in hs:
        input_set = data_generator.generate_points(h, h+n)
        experiments[h].run_from_input(n, input_set)
    if (n - start) % plot_step == 0:
        print(f"Finished running for n={n}")
        ExperimentalFramework.plot_multiple_experiments_min_time_ratios(list(experiments.values()), chans_algorithm, graham_scan_algorithm)

ExperimentalFramework.plot_multiple_experiments_min_time_ratios(list(experiments.values()), chans_algorithm, graham_scan_algorithm)

In [ ]:
# Standalone plot to see if h converges as n increases for generate_random_points_uniform
experiment_h_vs_n = ExperimentalFramework("Number of Hull Vertices vs Number of Points")
experiment_h_vs_n.set_independent_variable("Number of points")
experiment_h_vs_n.set_trials_per_run(100)

# we can use any algorithm here, we just want something to compute h
experiment_h_vs_n.add_algorithm("Number of Hull Vertices", jarvis_march) 

start = 2
end = 1_000_000
step = 10
data_generator.set_seed(42)
for n in range(start, end + step, step):
    experiment_h_vs_n.run_from_generator(n, data_generator, n)

    if (n - start) % 1000 == 0:
        print(f"Finished running for n={n}")
        experiment_h_vs_n.plot_boxplot_hull_vertices_vs_n(6)

experiment_h_vs_n.plot_boxplot_hull_vertices_vs_n(6)

In [ ]:
plt.rcParams["figure.figsize"] = (16, 10)
plt.rcParams['font.size'] = 20 

experiment_h_vs_n.plot_boxplot_hull_vertices_vs_n(6)